<a href="https://colab.research.google.com/github/diyanigam/Text-to-Website-CampEdUI/blob/main/CampEdUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import re
from collections import defaultdict
# cheatsheet contains all the components, tags and description
def parse_cheatsheet(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    components_by_package = defaultdict(list)
    tag_to_package = {}
    tag_descriptions = {}
    current_package = None
    current_desc = ""

    for line in lines:
        line = line.strip()

        if line.startswith("- @camped-ui/"):
            current_package = line.split()[0][2:]
        elif line.startswith("Description:"):
            current_desc = line.replace("Description:", "").strip()
        elif line.startswith("• <"):
            tag = re.findall(r"<(.*?)>", line)[0]
            components_by_package[current_package].append(tag)
            tag_to_package[tag] = current_package
            tag_descriptions[tag] = current_desc

    return components_by_package, tag_to_package, tag_descriptions

In [34]:
# tags present in the prompt
def infer_components_from_prompt(prompt, tag_to_package):
    used_tags = []
    for tag in tag_to_package:
        if tag.lower() in prompt.lower():
            used_tags.append(tag)
    return list(set(used_tags))


# import lines for tags
def generate_imports(used_tags, tag_to_package):
    pkg_to_tags = defaultdict(list)
    for tag in used_tags:
        pkg = tag_to_package.get(tag)
        if pkg:
            pkg_to_tags[pkg].append(tag)

    import_lines = []
    for pkg, tags in pkg_to_tags.items():
        line = f'import {{ {", ".join(sorted(set(tags))) } }} from "{pkg}";'
        import_lines.append(line)

    return "\n".join(import_lines)

In [35]:
def build_prompt(prompt_text, import_block):
    return f"""
### Instruction:
You are a Next.js code generator.

ONLY use components from Camped UI (https://ui.camped.academy/docs/components).
NEVER use raw HTML like <input>, <form>, or <button>.

Use Tailwind CSS for layout.
Output valid JSX code for a Next.js page.

For Example, to make a card component with name input field, a framework drop-down menu, deploy and cancel button, the following should be the output:
import {{ Button }} from "@camped-ui/button"
import {{
  Card,
  CardContent,
  CardDescription,
  CardFooter,
  CardHeader,
  CardTitle,
}} from "@camped-ui/card"
import {{ Input }} from "@camped-ui/input"
import {{ Label }} from "@camped-ui/label"
import {{
  Select,
  SelectContent,
  SelectItem,
  SelectTrigger,
  SelectValue,
}} from "@camped-ui/select"

export function CardWithForm() {{
  return (
    <Card className="w-[350px]">
      <CardHeader>
        <CardTitle>Create project</CardTitle>
        <CardDescription>Deploy your new project in one-click.</CardDescription>
      </CardHeader>
      <CardContent>
        <form>
          <div className="grid w-full items-center gap-4">
            <div className="flex flex-col space-y-1.5">
              <Label htmlFor="name">Name</Label>
              <Input id="name" placeholder="Name of your project" />
            </div>
            <div className="flex flex-col space-y-1.5">
              <Label htmlFor="framework">Framework</Label>
              <Select>
                <SelectTrigger id="framework">
                  <SelectValue placeholder="Select" />
                </SelectTrigger>
                <SelectContent position="popper">
                  <SelectItem value="next">Next.js</SelectItem>
                  <SelectItem value="sveltekit">SvelteKit</SelectItem>
                  <SelectItem value="astro">Astro</SelectItem>
                  <SelectItem value="nuxt">Nuxt.js</SelectItem>
                </SelectContent>
              </Select>
            </div>
          </div>
        </form>
      </CardContent>
      <CardFooter className="flex justify-between">
        <Button variant="outline">Cancel</Button>
        <Button>Deploy</Button>
      </CardFooter>
    </Card>
  )
}}

Here are the allowed imports:
{import_block}

### Prompt:
{prompt_text}

### Response:
""".strip()


In [36]:
# generate output

def run_generation(user_prompt, generator, components_by_package, tag_to_package):
    used_tags = infer_components_from_prompt(user_prompt, tag_to_package)
    if not used_tags:
        used_tags = ["Button", "Card", "CardContent", "CardHeader", "CardTitle", "Label", "Text"]

    import_block = generate_imports(used_tags, tag_to_package)
    full_prompt = build_prompt(user_prompt, import_block)
    output = generator(full_prompt, max_new_tokens=512, do_sample=True, temperature=0.7, return_full_text=False)
    return output[0]["generated_text"].strip()


In [37]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# loading model, tokeniser and text-generator
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/deepseek-coder-1.3b-instruct",
    trust_remote_code=True,
    load_in_8bit=True,
    device_map="auto",
    llm_int8_enable_fp32_cpu_offload=True
)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# loading cheatsheet
components_by_package, tag_to_package, tag_descriptions = parse_cheatsheet("cheatsheet.txt")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Device set to use cuda:0


In [ ]:
while True:
  user_input = input("Prompt: ")
  if user_input.lower() in ("exit"):
        break
  output = run_generation(user_input, generator, components_by_package, tag_to_package)
  print(output)


Prompt: help me make a card component with an input field,  password field, and a button that says login 
Sure, here is a simple example of a Card component with an Input field, Password field, and a Login button. You can use the same components as in your example.

```jsx
import { Button } from '@camped-ui/button'
import { Card, CardContent, CardFooter, CardHeader, CardTitle } from '@camped-ui/card'
import { Input } from '@camped-ui/input'
import { Label } from '@camped-ui/label'

export function CardWithForm() {
  return (
    <Card className="w-[350px]">
      <CardHeader>
        <CardTitle>Login</CardTitle>
      </CardHeader>
      <CardContent>
        <form>
          <div className="grid w-full items-center gap-4">
            <div className="flex flex-col space-y-1.5">
              <Label htmlFor="username">Username</Label>
              <Input id="username" placeholder="Your username" />
            </div>
            <div className="flex flex-col space-y-1.5">
            